Oracle Data Science service sample notebook.

Copyright (c) 2021-2022 Oracle, Inc.<br>
Licensed under the Universal Permissive License v 1.0 as shown at https://oss.oracle.com/licenses/upl.
</font>


# Deploy and Invoke a Model Using the Data Science Model Deployment
<p style="margin-left:10%; margin-right:10%;">by the <font color="teal">Oracle Cloud Infrastructure Data Science Service.</font></p>

***

## Overview

A common struggle that data scientists have is to get a model from the notebook into production. The Model Deployment service solves this problem by managing the deployment of models in the Model Catalog. This notebook uses the model that was previously stored in the Model Catalog and deploys it. The deployment is one or more compute instances, load balancers, and optionally a set of logs. The model deployment exposes an HTTPS REST API that allows applications to pass in observations and have the inferences returned.

***

**<font color='red'>NOTE: This notebook was run in the PySpark 3.0 and Data Flow (slug: `pyspark30_p37_cpu_v5`) conda environment.</font>**

***

Datasets are provided as a convenience. Datasets are considered third-party content and are not considered materials
under your agreement with Oracle.

You can access the `orcl_attrition` dataset license [here](https://oss.oracle.com/licenses/upl).

In [ ]:
import ads
import oci
import os
import requests

from ads.common.oci_logging import OCILogGroup, OCILog
from ads.model.deployment import ModelDeployer, ModelDeploymentProperties
from oci.signer import Signer

ads.set_auth(auth='resource_principal')

# Model OCID

In `1-model-training.ipynb` notebook, the model **attrition-model** was created and stored in the Model Catalog. This notebook will deploy that model to a REST endpoint where predictions can be made.

In the following cell, replace `<model_ocid>` in the `model_ocid` variable with the model OCID created in `1-model-training.ipynb` notebook. If you ran the notebook `1-model-training.ipynb`, copy the model OCID that is printed at the end of the notebook.

If the model OCID is not available from the `1-model-training.ipynb` notebook, it can be obtained from the Console. To do this, go to the Project and click on the project that contains the model. On the project details page, select the link **Models** on the menu on the left-hand side in the **Resources** section. Look for the model with the display name is **attrition-model**. Click on the **attrition-model** link. At the top of the screen look for the heading **OCID** and copy the OCID to the clipboard. Paste the OCID into the next cell, replacing `<model_ocid>` with the OCID. Make sure that the quotes (`'`) are left in place. The OCID will have **datasciencemodel** in the name. It will look something like `ocid1.datasciencemodel.oc1.iad.amaaaaaa...`.

In [ ]:
model_ocid = '<model_ocid>'
if model_ocid == '<model_ocid>':
        raise ValueError("Update the variable `model_ocid`.")

# Deploy the Model

The next cell specifies the project and compartment OCIDs. These will be used to specify what project and compartment the model deployment will use. 

In [ ]:
project_id = os.environ['PROJECT_OCID']
compartment_id = os.environ['NB_SESSION_COMPARTMENT_OCID']

Model deployments have the ability to capture information about the deployment through the use of two logs. The prediction log captures information that is printed to standard error and standard out. Thus, any information can be logged each time the endpoint is called. The content of the access log is generated each time a prediction is requested. It logs information such as the type of request (i.e. POST), latency, and other information that allows for the service to be monitored.

While logging is optional, it is highly recommended. Without logging enabled, it is very difficult to troubleshoot a model deployment or monitor its performance. The following cell will create a Log Group, and prediction and access logs. If you run this cell more than once you will have to change the names of the log group, and the logs' unique names.

In [ ]:
log_group = OCILogGroup(display_name="Attrition-Prediction-Deployment").create()
predict_log = log_group.create_log("Prediction")
access_log = log_group.create_log("Access")

The ADS SDK has a class called `ModelDeployer()`. The `.deploy()` method of this class accepts parameters that are needed for the model deployment. Some examples of this would be the model OCID that is to be used for inference, a display name so that the deployment can be located in the Console, the compute resources that are to be used, and the project and compartment OCIDs where the model deployment will be stored. The `.deploy()` method returns a `ModelDeployment` object. This object provides information about the model deployment and allows you to interact with it.

**The model deployment will take several minutes to complete** as it needs to provision and configure resources such as a load balancer, and the instances used in the deployment. ADS displays a progress bar to track the progress of your deployment. You can also keep track of the deployment in the OCI console. Once the model is deployed the model deployment OCID and the REST API endpoint are printed.

In [ ]:
deployer = ModelDeployer()

deployment = deployer.deploy(
    model_id=model_ocid,
    display_name="Attrition Prediction Deployment",
    instance_shape="VM.Standard2.1",
    instance_count=1,
    project_id=project_id,
    compartment_id=compartment_id,
    access_log_group_id = log_group.id,
    access_log_id = access_log.id,
    predict_log_group_id = log_group.id,
    predict_log_id = predict_log.id,
)

print(f"Model deployment OCID: {deployment.model_deployment_id}")
print(f"REST API end-point: {deployment.url}")

When a `ModelDeployment` object is printed, it displays detailed information about the model deployment. This information is also accessible through a family of accessor methods and attributes.

In [ ]:
deployment.properties

# Inference from the REST Endpoint

Client applications perform inference by sending REST API requests to the model deployment. Each model deployment has a unique URL and an endpoint where the request is sent. The default end-point for inference is `/predict`. Generally, the URL for the model deployment will have the form:

```
https://modeldeployment.<region>.oci.customer-oci.com/<model_deployment_ocid>
```
Therefore, the URL to send the inference requests to would be:
```
https://modeldeployment.<region>.oci.customer-oci.com/<model_deployment_ocid>/predict.
```

The payload that is sent to the inference endpoint is normally a JSON string. Each top-level key is the name of a parameter. The value is another dictionary where the key is the inference number and the value is the value to be used in the inference. For example,
```
"Age":{"0":42,"1":50,"2":38,"3":34,"5":33}
```

The parameter is `Age` the first observation (`"0"`) has a value of `42`. The second observation (`"1"`) has a value of `50`, etc.

The following cell defines the JSON payload that is passed to the model deployment end-point. This payload corresponds to the first five rows of the training dataset defined in notebook `1-model-training.ipynb`.

In [ ]:
input_data='''
{
 "Age":{"0":42,"1":50,"2":38,"3":34,"5":33},
 "TravelForWork":{"0":"infrequent","1":"often","2":"infrequent","3":"often","5":"often"},
 "SalaryLevel":{"0":5054,"1":1278,"2":6296,"3":6384,"5":4608},
 "JobFunction":{"0":"Product Management","1":"Software Developer","2":"Software Developer","3":"Software Developer","5":"Software Developer"},
 "CommuteLength":{"0":2,"1":9,"2":3,"3":4,"5":3},
 "EducationalLevel":{"0":"L2","1":"L1","2":"L2","3":"L4","5":"L2"},
 "EducationField":{"0":"Life Sciences","1":"Life Sciences","2":"Other","3":"Life Sciences","5":"Life Sciences"},
 "EnvironmentSatisfaction":{"0":2,"1":3,"2":4,"3":4,"5":4},
 "Gender":{"0":"Female","1":"Male","2":"Male","3":"Female","5":"Male"},
 "HourlyRate":{"0":94,"1":61,"2":92,"3":56,"5":79},
 "JobInvolvement":{"0":3,"1":2,"2":2,"3":3,"5":3},
 "JobRole":{"0":"Sales Executive","1":"Research Scientist","2":"Laboratory Technician","3":"Research Scientist","5":"Laboratory Technician"},
 "JobSatisfaction":{"0":4,"1":2,"2":3,"3":3,"5":4},
 "MaritalStatus":{"0":"Single","1":"Married","2":"Single","3":"Married","5":"Single"},
 "MonthlyRate":{"0":19479,"1":24907,"2":2396,"3":23159,"5":11864},
 "NumCompaniesWorked":{"0":8,"1":1,"2":6,"3":1,"5":0},
 "OverTime":{"0":"Yes","1":"No","2":"Yes","3":"Yes","5":"No"},
 "PercentSalaryHike":{"0":11,"1":23,"2":15,"3":11,"5":13},
 "PerformanceRating":{"0":3,"1":4,"2":3,"3":3,"5":3},
 "RelationshipSatisfaction":{"0":1,"1":4,"2":2,"3":3,"5":3},
 "StockOptionLevel":{"0":0,"1":1,"2":0,"3":0,"5":0},
 "YearsinIndustry":{"0":8,"1":10,"2":7,"3":8,"5":8},
 "TrainingTimesLastYear":{"0":0,"1":3,"2":3,"3":3,"5":2},
 "WorkLifeBalance":{"0":1,"1":3,"2":3,"3":3,"5":2},
 "YearsAtCurrentLevel":{"0":4,"1":7,"2":0,"3":7,"5":7},
 "YearsSinceLastPromotion":{"0":0,"1":1,"2":0,"3":3,"5":3},
 "name":{"0":"Tracy Moore","1":"Andrew Hoover","2":"Julie Bell","3":"Thomas Adams","5":"Rhonda Grant"}
}
'''

## Inference Using ModelDeployment Object

A `ModelDeployment` object has a `.predict()` method that takes a JSON string of values that are to be used for inference. It is convenient in that it determines the REST endpoint and it handles the authentication.

When a model is initially deployed you will have access to its `ModelDeployment` object. However, the notebook that did the model deployment is quite often different from the notebook that is going to make inferences. The `.get_model_deployment()` method in the `ModelDeployer` class accepts a model deployment OCID and returns a `ModelDeployment` object. This object can be used for inference.

The following cell uses this technique to make inferences.

In [ ]:
print(f"Getting ModelDeployment object for OCID: {deployment.model_deployment_id}")
deployment_from_ocid = ModelDeployer().get_model_deployment(deployment.model_deployment_id)

This `ModelDeployment` object can be used for model inference using the `.predict()` method.

In [ ]:
deployment_from_ocid.predict(input_data)

## Inference Using REST Calls

Often a model deployment needs to be accessed from applications that not using the ADS Python library or do not have access to create a `ModelDeployment` object using a Model Deployment OCID. This is not an issue as the Model Deployment service uses standard POST REST API calls. If ADS is not being used, then the client application must determine the REST prediction end-point and handle authentication. As discussed above, the REST API endpoint for inference is the model deployment's URL followed by `/predict`. Authentication can use API keys or a resource principal.

The next cell creates the URL for the REST call. It also obtains authentication using a resource principal. The Python package, `requests`, is used to make an HTTPS REST API call.

In [ ]:
uri = deployment.url + "/predict"
print(f"Sending inference request to: {uri}")
auth = oci.auth.signers.get_resource_principals_signer()
requests.post(uri, json=input_data, auth=auth).json()

# Predict from Console Model Deployment (Optional)

In Lab 7, Step 5 there are instructions to manually create a model deployment from using the Console. This section calls the HTTPS REST API endpoint for that model deployment.

The following cell needs to be updated with the model deployment endpoint URL. Replace `<model_deployment_url>` with the URL from Step 5 of Lab 7. It should look something like this:

```
https://modeldeployment.<region>.oci.customer-oci.com/ocid1.datasciencemodeldeployment.oc1.iad.amaaaaaa.../predict
```

In [ ]:
md_url = '<model_deployment_url>'

In [ ]:
if md_url == '<model_deployment_url>':
        print("Update the variable `md_url` to perform inference on the model deployment created in the Console.")
else:
    print(f"Sending inference request to: {md_url}")
    auth = oci.auth.signers.get_resource_principals_signer()
    print(requests.post(md_url, json=input_data, auth=auth).json())